In [1]:
from IPython.core.display import display, HTML, Javascript
from string import Template
import json, os
import pandas as pd
import numpy as np

In [2]:
notebook_path = os.path.abspath("Shooting_VS_Income.ipynb")
historic_csv = os.path.join(os.path.dirname(notebook_path), "data/", "historic_incident_with_zipcode_cleaned.csv")
historic = pd.read_csv(historic_csv)
current_csv = os.path.join(os.path.dirname(notebook_path), "data/", "historic_incident_with_zipcode2020_cleaned.csv")
current = pd.read_csv(current_csv)
income_csv = os.path.join(os.path.dirname(notebook_path), "data/", "Median_Income_Byzip_Avg.csv")
income = pd.read_csv(income_csv)
current.head()

,Unnamed: 0,INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,JURISDICTION_CODE,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,ZipCode
0,5,214607781,06/25/2020,19:58:00,MANHATTAN,0,M,WHITE HISPANIC,18-24,10027
1,11,208237500,01/20/2020,04:50:00,MANHATTAN,0,M,BLACK HISPANIC,45-64,10467
2,12,214513453,06/23/2020,19:15:00,STATEN ISLAND,0,M,WHITE HISPANIC,<18,10472
3,16,217256574,08/31/2020,14:55:00,BRONX,0,M,BLACK,25-44,10470
4,18,213972320,06/08/2020,15:00:00,QUEENS,0,M,BLACK HISPANIC,45-64,10032


In [3]:
bigdata = historic.append(current, ignore_index=True)

In [4]:
bigdata['YEAR'] = pd.DatetimeIndex(bigdata['OCCUR_DATE']).year
filtered_data = bigdata.loc[(bigdata['YEAR'] <= 2019) & (bigdata['YEAR'] >= 2017)]
grpby = filtered_data.groupby('ZipCode').size().reset_index(name='counts')
grpby.head()

,ZipCode,counts
0,10001,2
1,10002,5
2,10003,1
3,10004,1
4,10009,4


In [5]:
shooting_income =  pd.merge(income, grpby, on='ZipCode', how='left')
shooting_income['counts'] = shooting_income['counts'].fillna(0)
shooting_income['counts']=shooting_income['counts'].astype('int')
data = shooting_income.to_dict(orient='records')
#shooting_income.to_csv('shooting_vs_income.csv')
#print(data)

In [6]:
%%javascript
require.config({
    paths: {
        d3: "https://d3js.org/d3.v6.min"
     }
});

require(["d3"], function(d3) {
    console.log(d3);
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [7]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')

In [8]:
css_text = ''''''

In [9]:
js_text_template = Template('''
  var margin = ({top: 20, bottom: 40, left: 50, right: 20})  ;
  var visWidth = 900 - margin.left - margin.right;
  var visHeight = 500 - margin.top - margin.bottom;
  
  var data = $python_data;
  
  var maxCount = d3.max(data, d => d.counts);
  var maxIncome = d3.max(data, d => d.Median_Household_Income);
  
  var countScale = d3.scaleLinear()
    .domain([0, maxCount])
    .range([visHeight, 0])
    .nice()
  
  var incomeScale = d3.scaleLinear()
    .domain([0, maxIncome])
    .range([0, visWidth])
    .nice()
  
  var xAxis = d3.axisBottom(incomeScale) 
     .tickFormat(d => d/1000 + 'k');
  
  var yAxis = d3.axisLeft(countScale);
  
  var svg = d3.select("#$graphdiv")
      .append('svg')
      .attr('width', visWidth + margin.left + margin.right)
      .attr('height', visHeight + margin.top + margin.bottom); 
  
  var g = svg.append("g")
      .attr('transform', `translate(${margin.left}, ${margin.top})`);  
  
  g.selectAll('circle')
    .data(data)
    .join('circle')
      .attr('cx', d => incomeScale(d.Median_Household_Income))
      .attr('cy', d => countScale(d.counts))
      .attr('r', 7)
      .attr('fill', 'cornflowerblue')  
  
  g.append('g')
      .call(yAxis);  
  
  g.append('g')
      .attr('transform', `translate(0, ${visHeight})`)
      .call(xAxis)  
  
  g.append('text')
      .attr('fill', 'black')
      .attr('font-family', 'sans-serif')
      .attr('x', visWidth / 8)
      .attr('y', 0)
      .style('font-size', 18)
      .text("Median Household Income vs Number of Gun Violence (2017-2019)");
  
  g.append('text')
      .attr('fill', 'black')
      .attr('font-family', 'sans-serif')
      .attr('x', visWidth / 3)
      .attr('y', 20)
      .style('font-size', 18)
      .text("by Postal Code Area");  
''')

In [10]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')
js_text = js_text_template.safe_substitute(python_data=json.dumps(data),
                                       graphdiv='graph-div')
HTML(html_template.safe_substitute(css_text=css_text, js_text=js_text))